# **Improving Employee Retention by Predicting Employee Attrition Using Machine Learning**

# **Load Dataset**
---

In [1]:
# import library
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

In [2]:
# read dataset
read = pd.read_csv('hr_data.csv')

In [3]:
df = read.copy()

# **Overview Dataset**
---

### Check Shape and Datatype

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287 entries, 0 to 286
Data columns (total 25 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Username                            287 non-null    object 
 1   EnterpriseID                        287 non-null    int64  
 2   StatusPernikahan                    287 non-null    object 
 3   JenisKelamin                        287 non-null    object 
 4   StatusKepegawaian                   287 non-null    object 
 5   Pekerjaan                           287 non-null    object 
 6   JenjangKarir                        287 non-null    object 
 7   PerformancePegawai                  287 non-null    object 
 8   AsalDaerah                          287 non-null    object 
 9   HiringPlatform                      287 non-null    object 
 10  SkorSurveyEngagement                287 non-null    int64  
 11  SkorKepuasanPegawai                 282 non-n

In [5]:
# jumlah record dan feature
df.shape

(287, 25)

### Check Duplicate

In [6]:
# cek duplikat data
df.duplicated().sum()

0

### Check Null Value

In [22]:
# melihat ringkasan data
list_item = []
for col in df.columns:
    list_item.append([col, df[col].isna().sum(), round(100*df[col].isna().sum()/len(df[col]), 2), df[col].nunique(), df[col].unique()[:10]])

df_desc = pd.DataFrame(data=list_item, columns='feature, null_values, null_percentage, unique_values, unique_sample'.split(","))
df_desc

,feature,null_values,null_percentage,unique_values,unique_sample
0,Username,0,0.00,285,"[spiritedPorpoise3, jealousGelding2, pluckyMue..."
1,EnterpriseID,0,0.00,287,"[111065, 106080, 106452, 106325, 111171, 10641..."
2,StatusPernikahan,0,0.00,5,"[Belum_menikah, Menikah, Bercerai, Lainnya, -]"
3,JenisKelamin,0,0.00,2,"[Pria, Wanita]"
4,StatusKepegawaian,0,0.00,3,"[Outsource, FullTime, Internship]"
5,Pekerjaan,0,0.00,14,"[Software Engineer (Back End), Data Analyst, S..."
6,JenjangKarir,0,0.00,3,"[Freshgraduate_program, Senior_level, Mid_level]"
7,PerformancePegawai,0,0.00,5,"[Sangat_bagus, Sangat_kurang, Bagus, Biasa, Ku..."
8,AsalDaerah,0,0.00,5,"[Jakarta Timur, Jakarta Utara, Jakarta Pusat, ..."
9,HiringPlatform,0,0.00,9,"[Employee_Referral, Website, Indeed, LinkedIn,..."


In [21]:
df['IkutProgramLOP'].value_counts()

1.0    15
0.0    14
Name: IkutProgramLOP, dtype: int64

terlalu banyak null, fitur tidak dipakai

In [43]:
nan_in_col  = df[df['AlasanResign'].isna()]
is_resign = nan_in_col[['AlasanResign', 'TanggalResign']]
is_resign['TanggalResign'].value_counts()

-    66
Name: TanggalResign, dtype: int64

null value pada AlasanResign memiliki null value pada TanggalResign -> asumsi bahwa karyawan masih bekerja

In [46]:
uiux_in_col  = df[df['AlasanResign'] == 'Product Design (UI & UX)']
uiux_in_col.T

,27,124,147,203
Username,pluckyApples4,dearKitten3,brainyLlama9,hushedBobolink8
EnterpriseID,110566,111266,100884,106392
StatusPernikahan,Menikah,Belum_menikah,Menikah,Menikah
JenisKelamin,Wanita,Pria,Wanita,Wanita
StatusKepegawaian,FullTime,FullTime,FullTime,FullTime
Pekerjaan,Software Engineer (Front End),Product Design (UI & UX),Software Engineer (Front End),Software Engineer (Back End)
JenjangKarir,Senior_level,Freshgraduate_program,Freshgraduate_program,Mid_level
PerformancePegawai,Biasa,Sangat_bagus,Sangat_bagus,Sangat_bagus
AsalDaerah,Jakarta Selatan,Jakarta Utara,Jakarta Timur,Jakarta Selatan
HiringPlatform,LinkedIn,Diversity_Job_Fair,LinkedIn,Website


In [48]:
df['Pekerjaan'].value_counts()

Software Engineer (Back End)      109
Software Engineer (Front End)      72
Software Engineer (Android)        24
Product Design (UI & UX)           24
Product Manager                    17
Data Analyst                       16
Data Engineer                      10
Scrum Master                        3
Software Engineer (iOS)             3
DevOps Engineer                     3
Digital Product Manager             2
Machine Learning Engineer           2
Product Design (UX Researcher)      1
Software Architect                  1
Name: Pekerjaan, dtype: int64

Value tersebut ambigu -> ganti ke 'lainnya'

In [60]:
df['JumlahKetidakhadiran'].value_counts()

4.0     20
2.0     20
15.0    19
7.0     17
16.0    17
6.0     16
14.0    16
3.0     16
19.0    15
13.0    15
17.0    14
1.0     14
20.0    13
9.0     12
11.0    12
10.0    10
8.0     10
5.0      9
12.0     7
18.0     7
50.0     1
55.0     1
Name: JumlahKetidakhadiran, dtype: int64

In [57]:
df['SkorKepuasanPegawai'].value_counts()

3.0    96
5.0    91
4.0    85
2.0     8
1.0     2
Name: SkorKepuasanPegawai, dtype: int64

In [59]:
df['JumlahKeikutsertaanProjek'].value_counts()

0.0    221
6.0     20
5.0     20
7.0      9
4.0      8
3.0      3
1.0      1
2.0      1
8.0      1
Name: JumlahKeikutsertaanProjek, dtype: int64

In [61]:
df['JumlahKeterlambatanSebulanTerakhir'].value_counts()

0.0    255
4.0      8
3.0      6
2.0      6
5.0      5
6.0      5
1.0      1
Name: JumlahKeterlambatanSebulanTerakhir, dtype: int64

### Check Inconsistency Values

In [24]:
# melihat jumlah dan nilai unik kolom kategorikal

feats = ['StatusPernikahan', 'JenisKelamin', 'StatusKepegawaian', 'Pekerjaan', 'JenjangKarir', 'PerformancePegawai', 
        'AsalDaerah', 'HiringPlatform', 'TingkatPendidikan', 'PernahBekerja', 'AlasanResign']

for i in feats:
    unique_values = df[i].value_counts()
    print(f"{i}\n{unique_values}\n")
    print('-'*50)

StatusPernikahan
Belum_menikah    132
Menikah           57
Lainnya           48
Bercerai          47
-                  3
Name: StatusPernikahan, dtype: int64

--------------------------------------------------
JenisKelamin
Wanita    167
Pria      120
Name: JenisKelamin, dtype: int64

--------------------------------------------------
StatusKepegawaian
FullTime      217
Outsource      66
Internship      4
Name: StatusKepegawaian, dtype: int64

--------------------------------------------------
Pekerjaan
Software Engineer (Back End)      109
Software Engineer (Front End)      72
Software Engineer (Android)        24
Product Design (UI & UX)           24
Product Manager                    17
Data Analyst                       16
Data Engineer                      10
Scrum Master                        3
Software Engineer (iOS)             3
DevOps Engineer                     3
Digital Product Manager             2
Machine Learning Engineer           2
Product Design (UX Researcher)     

# **Cleaning Dataset**
---

### Handling Null Values

In [25]:
df_clean = df.copy()

In [26]:
# PernahBekerja
df_clean.drop(columns = 'PernahBekerja', inplace=True)

# IkutProgramLOP
df_clean.drop(columns = 'IkutProgramLOP', inplace=True)

# AlasanResign 
df_clean['AlasanResign'].fillna('masih_bekerja', inplace=True)

# JumlahKetidakhadiran 
df_clean['JumlahKetidakhadiran'].fillna(0, inplace=True)

# JumlahKeikutsertaanProjek 
df_clean['JumlahKeikutsertaanProjek'].fillna(0, inplace=True)

# JumlahKeterlambatanSebulanTerakhir 
df_clean['JumlahKeterlambatanSebulanTerakhir'].fillna(0, inplace=True)

# SkorKepuasanPegawai 
df_clean['SkorKepuasanPegawai'].fillna(df_clean['SkorKepuasanPegawai'].median(), inplace=True)

In [27]:
df_clean.isna().sum().sum()

0

### Handling Inconsistency Values

In [28]:
# AlasanResign 
df_clean['AlasanResign'].replace(['Product Design (UI & UX)'], 'lainnya', inplace=True)

# StatusPernikahan
df_clean['StatusPernikahan'].replace(['-'], 'Lainnya', inplace=True)

# HiringPlatform
df_clean['HiringPlatform'].replace(['On-line_Web_application'], 'Website', inplace=True)

In [29]:
# cek value
df_clean['AlasanResign'].value_counts()

masih_bekerja        198
jam_kerja             16
ganti_karir           14
kejelasan_karir       11
tidak_bisa_remote     11
toxic_culture         10
leadership             9
tidak_bahagia          8
internal_conflict      4
lainnya                4
apresiasi              2
Name: AlasanResign, dtype: int64

In [30]:
# cek value
df_clean['StatusPernikahan'].value_counts()

Belum_menikah    132
Menikah           57
Lainnya           51
Bercerai          47
Name: StatusPernikahan, dtype: int64

In [31]:
# cek value
df_clean['HiringPlatform'].value_counts()

Indeed                85
LinkedIn              69
Google_Search         46
Diversity_Job_Fair    27
Employee_Referral     24
CareerBuilder         22
Website               12
Other                  2
Name: HiringPlatform, dtype: int64

### Handling Datatype

In [32]:
feats_int = ['JumlahKetidakhadiran', 'JumlahKeikutsertaanProjek', 'JumlahKeterlambatanSebulanTerakhir', 'SkorKepuasanPegawai']

# convert to integer
df_clean[feats_int] = df_clean[feats_int].astype('int64')

In [35]:
# cek
df_clean[feats_int].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287 entries, 0 to 286
Data columns (total 4 columns):
 #   Column                              Non-Null Count  Dtype
---  ------                              --------------  -----
 0   JumlahKetidakhadiran                287 non-null    int64
 1   JumlahKeikutsertaanProjek           287 non-null    int64
 2   JumlahKeterlambatanSebulanTerakhir  287 non-null    int64
 3   SkorKepuasanPegawai                 287 non-null    int64
dtypes: int64(4)
memory usage: 9.1 KB


In [45]:
feats_dt = ['TanggalLahir', 'TanggalHiring', 'TanggalPenilaianKaryawan', 'TanggalResign']

for i in feats_dt:
    df_clean[i] = pd.to_datetime(df_clean[i], errors='coerce')

In [46]:
df_clean[feats_dt].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287 entries, 0 to 286
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   TanggalLahir              287 non-null    datetime64[ns]
 1   TanggalHiring             287 non-null    datetime64[ns]
 2   TanggalPenilaianKaryawan  287 non-null    datetime64[ns]
 3   TanggalResign             89 non-null     datetime64[ns]
dtypes: datetime64[ns](4)
memory usage: 9.1 KB


### Additional

In [48]:
# convert TanggalLahir to Age fitur
# create function for calculate age
from datetime import date
def calculate_age(birthdate):
    today = date.today()
    age = today.year-birthdate.year-((today.month, today.day) < (birthdate.month, birthdate.day))
    return age

# fit into dataframe
df_clean['Age'] = df_clean['TanggalLahir'].apply(calculate_age)
df_clean.drop(columns='TanggalLahir', inplace=True)

In [49]:
# drop fitur konstanta
df_clean.drop(columns = ['Username', 'EnterpriseID', 'NomorHP', 'Email'], inplace=True)

In [50]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287 entries, 0 to 286
Data columns (total 19 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   StatusPernikahan                    287 non-null    object        
 1   JenisKelamin                        287 non-null    object        
 2   StatusKepegawaian                   287 non-null    object        
 3   Pekerjaan                           287 non-null    object        
 4   JenjangKarir                        287 non-null    object        
 5   PerformancePegawai                  287 non-null    object        
 6   AsalDaerah                          287 non-null    object        
 7   HiringPlatform                      287 non-null    object        
 8   SkorSurveyEngagement                287 non-null    int64         
 9   SkorKepuasanPegawai                 287 non-null    int64         
 10  JumlahKeikutsertaanProjek 

# **EDA**
---